# Patient Identifiers

## Import and setup

In [15]:
import requests

rest_url = 'http://localhost:8080/openmrs/ws/rest/v1/'
myauth = ('admin', 'Admin123')

## Identifier Types

In [ ]:
id_types: dict = requests.get(rest_url + 'patientidentifiertype', auth=myauth).json()
for res in id_types.get('results'):
    print(f'{res.get('uuid')}: {res.get('display')}')

05a29f94-c0ed-11e2-94be-8c13b969e334: OpenMRS ID
b4143563-16cd-4439-b288-f83d61670fc8: ID Card
22348099-3873-459e-a32e-d93b17eda533: Legacy ID
8d79403a-c2cc-11de-8d13-0010c6dffd0f: Old Identification Number
8d793bee-c2cc-11de-8d13-0010c6dffd0f: OpenMRS Identification Number
a71403f3-8584-4289-ab41-2b4e5570bd45: SSN


In [18]:
nic_payload = {
    'name': 'NADRA NIC Number',
    'description': 'National Identity Card granted by NADRA',
    'format': '\\d{5}-\\d{7}-\\d{1}',
    'formatDescription': 'Placeholder: 12345-1234567-1',
    'required': False,
}
nic_resp = requests.post(rest_url + 'patientidentifiertype', json = nic_payload, auth = myauth)
nic_resp.json()

{'uuid': '31f70925-2606-468b-84fb-2d756f4fc083',
 'display': 'NADRA NIC Number',
 'name': 'NADRA NIC Number',
 'description': 'National Identity Card granted by NADRA',
 'format': '\\d{5}-\\d{7}-\\d{1}',
 'formatDescription': 'Placeholder: 12345-1234567-1',
 'required': False,
 'validator': None,
 'locationBehavior': None,
 'uniquenessBehavior': None,
 'retired': False,
 'links': [{'rel': 'self',
   'uri': 'http://localhost:8080/openmrs/ws/rest/v1/patientidentifiertype/31f70925-2606-468b-84fb-2d756f4fc083',
   'resourceAlias': 'patientidentifiertype'},
  {'rel': 'full',
   'uri': 'http://localhost:8080/openmrs/ws/rest/v1/patientidentifiertype/31f70925-2606-468b-84fb-2d756f4fc083?v=full',
   'resourceAlias': 'patientidentifiertype'}],
 'resourceVersion': '2.0'}

## Identifier Sources

In [ ]:
id_sources: dict = requests.get(rest_url + 'idgen/identifiersource', auth = myauth).json()
oid_gen = id_sources.get('results')[0]

f'{oid_gen.get('uuid')}: {oid_gen.get('display')}'

'8549f706-7e85-4c1d-9424-217d50a2988b: OpenMRS ID - Generator for OpenMRS ID - org.openmrs.module.idgen.SequentialIdentifierGenerator'

### Using an identifier source to generate an identifier

In [31]:
id_generation = {
    'generateIdentifiers': True,
    'sourceUuid': '8549f706-7e85-4c1d-9424-217d50a2988b',
    'numberToGenerate': 3
}
new_id = requests.post(rest_url + 'idgen/identifiersource', json = id_generation, auth = myauth)
new_id.json()

{'identifiers': ['10001YY', '100020W', '100021U']}

## Creating a patient

### 1. Pick the type and source

In [ ]:
target_type = 'OpenMRS ID'

id_sources: dict = requests.get(rest_url + 'idgen/identifiersource', auth = myauth).json()
source_uuid = id_sources.get('results')[0].get('uuid')

target_source: dict

for each in id_sources.get('results'):
    if target_type in each.get('display'):
        target_source = each

id_source_uuid = each.get('uuid')
id_source_name = each.get('display')
id_type_uuid = each.get('identifierType').get('uuid')
id_type_name = each.get('identifierType').get('display')

print(f"ID Source UUID: {id_source_uuid}")
print(f"ID Source: {id_source_name}")
print(f"ID Type UUID: {id_type_uuid}")
print(f"ID Type: {id_type_name}")

ID Source UUID: 8549f706-7e85-4c1d-9424-217d50a2988b
ID Source: OpenMRS ID - Generator for OpenMRS ID - org.openmrs.module.idgen.SequentialIdentifierGenerator
ID Type UUID: 05a29f94-c0ed-11e2-94be-8c13b969e334
ID Type: OpenMRS ID


### 2. Generate ID from source

In [45]:
idgen_payload = {
    'generateIdentifiers': True,
    'sourceUuid': source_uuid,
    'numberToGenerate': 1
}
idgen_response: dict = requests.post(rest_url + 'idgen/identifiersource', json = idgen_payload, auth = myauth).json()
new_id = idgen_response.get('identifiers')[0]

new_id

'100025J'

### 3. Create patient with new ID

In [46]:
new_patient_payload = {
    'identifiers': [
        {
            'identifier': new_id,
            'identifierType': id_type_uuid,
            'preferred': False
        }
    ],
    'person': {
        'gender': 'F',
        'age': 47,
        'birthdate': '1970-01-01T00:00:00.000+0100',
        'birthdateEstimated': False,
        'dead': False,
        'deathDate': None,
        'causeOfDeath': None,
        'names': [{'givenName': 'Python', 'familyName': 'Script'}],
        'addresses': [
            {
                'address1': 'Plot number 33, Malir 15',
                'cityVillage': 'Karachi',
                'country': 'Pakistan',
                'postalCode': '75210'
            }
        ]
    },
}
patient_creation = requests.post(rest_url + 'patient', json = new_patient_payload, auth = myauth)
patient_creation.json()

{'uuid': '8cd1a0a8-3782-442c-ae34-14a84e2efc22',
 'display': '100025J - Python Script',
 'identifiers': [{'uuid': 'b70738c2-6437-45be-a2b7-7389e337d6c0',
   'display': 'OpenMRS ID = 100025J',
   'links': [{'rel': 'self',
     'uri': 'http://localhost:8080/openmrs/ws/rest/v1/patient/8cd1a0a8-3782-442c-ae34-14a84e2efc22/identifier/b70738c2-6437-45be-a2b7-7389e337d6c0',
     'resourceAlias': 'identifier'}]}],
 'person': {'uuid': '8cd1a0a8-3782-442c-ae34-14a84e2efc22',
  'display': 'Python Script',
  'gender': 'F',
  'age': 55,
  'birthdate': '1970-01-01T04:00:00.000+0500',
  'birthdateEstimated': False,
  'dead': False,
  'deathDate': None,
  'causeOfDeath': None,
  'preferredName': {'uuid': 'fc6509b4-9650-4916-a704-bdb52978c507',
   'display': 'Python Script',
   'links': [{'rel': 'self',
     'uri': 'http://localhost:8080/openmrs/ws/rest/v1/person/8cd1a0a8-3782-442c-ae34-14a84e2efc22/name/fc6509b4-9650-4916-a704-bdb52978c507',
     'resourceAlias': 'name'}]},
  'preferredAddress': {'uui